# LSTM text generation
https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py

In [18]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 600901
total chars: 59


In [19]:
len(text)

600901

## Use our data set

Some IMDB movie comments

In [20]:
text_p = open('data/pos.txt').read().lower()
text_n = open('data/neg.txt').read().lower()
print(len(text_p))
print(len(text_n))

626349
612459


In [21]:
text_p[0:300]

'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . \nthe gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequ'

In [22]:
import random

text_lines = []

with open('data/pos.txt') as f:
    for line in f:
        text_lines.append(line)
with open('data/neg.txt') as f:
    for line in f:
        text_lines.append(line)
        
random.shuffle(text_lines)

In [23]:
text_lines[0:3]

['its screenplay serves as auto-critique , and its clumsiness as its own most damning censure . \n',
 "if we're to slap protagonist genevieve leplouff because she's french , do we have that same option to slap her creators because they're clueless and inept ? \n",
 'elegantly crafted but emotionally cold , a puzzle whose intricate construction one can admire but is difficult to connect with on any deeper level . \n']

In [24]:
text = ''.join(text_lines)

print(text[0:300])

its screenplay serves as auto-critique , and its clumsiness as its own most damning censure . 
if we're to slap protagonist genevieve leplouff because she's french , do we have that same option to slap her creators because they're clueless and inept ? 
elegantly crafted but emotionally cold , a puzz


## Check size of our dataset:

In [25]:
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 1238808
total chars: 91


## Run the model:

In [26]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

nb sequences: 412923
Vectorization...
Build model...


In [61]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

In [ ]:
# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [85]:
def sample_demo(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    print(preds)
    return np.argmax(probas)

print(sample_demo([0.7, 0.2, 0.1], temperature=0.1))
print(sample_demo([0.7, 0.2, 0.1], temperature=1.0))
print(sample_demo([0.7, 0.2, 0.1], temperature=5.0))

[  9.99996371e-01   3.62508322e-06   3.54012033e-09]
0
[ 0.7  0.2  0.1]
1
[ 0.4071692   0.31692851  0.27590229]
0
